The unit root is a combination of autoregressive coefficients of a series that indicates non-stationarity if it equals an integer value. 

Unit roots in time series apply to all variations of ARIMA models. A unit root in a time series indicates that the stochastic process cannot be stationary. The order of the unit root indicates the differencing required to produce a stationary process, where the unit root is solved by an equation involving a number m and autoregressive coefficients of the series. Equation similar to solving for polynomial roots, and if a solution exists then unit root of order m must be differenced m times to produce stationary series. 

This can be checked by taking unit root and inputting it into expectation and variance formulas for a series, which would result in changing variance and therefore indicate the series is non-stationary.

Tests for stationarity; can use augmented Dickey-Fuller Test which uses a null hypothesis to return a p-value to check whether hypothesis holds or not. The null hypothesis checks whether a unit root exists in the time series. If it does then lagged values cannot be used for predicting future time steps therefore the time-series isn't stationary. If the null hypothesis does not hold (it is false) then the opposite must occur and the time series is stationary.

Manually check test-statistic against Dickey-Fuller T-distribution; however stats packages can do this automatically

Granger-Causality test determines if one time series useful in forecasting another. Difficult to determine if changes in one series are correlated to changes in another series over large time (small correlations between series however are easier to observe). However correlation does not imply causality; better to describe test as whether time series X can be used to forecast time series Y (not necessarily CAUSE changes in time series Y).

Null hypothesis states that no lagged values of series X can be used to predict series Y with significant improvement; proving it wrong means series X can be used to forecast series Y however. 

For each lag, run it through t-test to determine if lags of x can be used to predict y. Then run all combined lags through an f-test to determine if combination of all lags of x can be used to predict y. 

Often time series that work under Granger Causality are those that have a similar form and are shifted version of one another (time lags from series X can be used to forecast Y given series Y is a shifted version of series X)

Granger-Causality can be used in economics to understand how mimicking export patterns of successful geographies can elevate & improve less successful geographies by assuming some relationship exists between both economies.

Evaluation metrics used for model selection include: Akaike Information Criterion (AIC) and Bayesian Information Criterion (BIC). 

AIC evaluates a collection of models and estimates the quality of each model relative to others; penalties are provided for number of parameters to reduce overfitting. 

BIC very similar to AIC however it utilizes a Bayesian approach. 

Generally hypothesis testing can be done using AIC rather than test statistics and p-values given that each test compares two statistical models. AIC is a generalist approach that differs from frequentist and bayesian statistics. 